In [ ]:
#%reset -f
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np
import feather
import time

import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
import h2oaiglm as h2oaiglm

### Import Data Frame and create raw X and y arrays

In [7]:
t0 = time.time()
df = pd.read_csv("./ipums.txt", sep=" ")
t1 = time.time()
print("Time to read data via feather: %r" % (t1-t0))

Please download https://drive.google.com/file/d/0By-R0tLVMSykckNocHZPeEN1VU0/view?usp=sharing
Time to read data via feather: 1.9766337871551514


In [8]:
target = df.columns[-1] ## last column is the response
cols = [c for c in df.columns if c != target]

In [9]:
X = np.array(df.ix[:,cols])
y = df[target].values
print(X.shape)
print(y.shape)

(999, 9732)
(999,)


### H2O AI GLM using the GPU Pointers

In [16]:
sourceDev = 0
gpu=True
nGPUs = 2
intercept = 1
lambda_min_ratio = 1e-4
nLambdas = 100
nAlphas = 1

H = (int)(0.8*X.shape[0])

trainX = X[:H,:]
trainY = y[:H]

validX = X[H:,:]
validY = y[H:]

## TODO: compute these in C++ (CPU or GPU)
lambda_max0 = max(abs(trainX.T.dot(trainY)))
sdTrainY = np.sqrt(np.var(trainY))
meanTrainY = np.mean(trainY)
mTrain = trainX.shape[0]
mValid = validX.shape[0]
n = trainX.shape[1]
print(mTrain)
print(mValid)
print(n)

sdValidY = np.sqrt(np.var(validY))
meanValidY = np.mean(validY)

799
200
9732


In [13]:
Solver = h2oaiglm.ElasticNetSolverGPU if gpu else h2oaiglm.ElasticNetSolverCPU
enet = Solver(nGPUs, 'r', intercept, 0, lambda_min_ratio, nLambdas, nAlphas)

In [14]:
a,b,c,d = enet.upload_data(sourceDev, trainX, trainY, validX, validY)

c_void_p(1112589467648)
c_void_p(1112652382208)
c_void_p(1112654479360)
c_void_p(1112671256576)


In [17]:
t0 = time.time()
enet.fit(sourceDev, mTrain, n, mValid, lambda_max0, sdTrainY, meanTrainY, sdValidY, meanValidY, a, b, c, d)
t1 = time.time()
print("Time to train H2O AI GLM: %r" % (t1-t0))

Time to train H2O AI GLM: 0.8264780044555664
